In [37]:
import math
import os
import random
import sys
import struct
import time
import warnings

import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt 


# nb 108: WIP hash function

We want a hash mapping range $(10, 400, 352)$ to integers in the range $(0, K)$.

First, convert a tuple $(i, j, k)$ to $x$ as $i*400*352 + j*352 + k$. This gives us integers in range $(10 * 400 * 352)$.

Now, there are all sorts of hash functions. Let's use the equidistribution theorem, that is, we assume $n\cdot \alpha \mod 1$ is equally distributed from $[0, 1)$ over integers $n$ for irrational $\alpha$. We'll use the golden ratio here.

With equidistribution, we map $(i, j, k)$ to $x$ to $[0, 1)$ to $[0, K)$ (integer) to $[0, 10 * 400 * 352)$ to $(i, j, k)$.

> *Note that $K$ is unrelated to $k$. This is silly but I used $k$ and $K$ too much to change it now.

So, this is:

1. $(i, j, k) \to x$ with `np.ravel_multi_index((i, j, k), (D, H, W))`.
2. $x -> q$ with the equidistribution hash. Note $q \in [0, K)$.
3. Then, map $q -> x$ by keeping track of indices.
4. $x \to (i, j, k)$ with `np.unravel_index(x, (10, 400, 352))`.

In [8]:
# map xx to i, j, k
np.unravel_index(354, (10, 400, 352))

(0, 1, 2)

In [10]:
# map i, j, k to xx
np.ravel_multi_index((0, 1, 2), (10, 400, 352))

354

In [11]:
#PHI = (1 + 5 ** 0.5) / 2 = 1.618033988749895

def equi_hash(n, a = 0.618033988749895, K = 131072):
    """Hash integer n to range [0, K) using Equidistribution Theorem Hash, using irrational a.

    Is optimal for a = Phi = 1.618034.
    Because of mod 1, we use 0.618034.

    :param n: Input to hash
    :type n: int
    :param a: Irrational number, defaults to 0.618033988749895
    :type a: float, optional
    :param K: [description], defaults to 131072
    :type K: int, optional

    :return: Result of the hash: An int in range [0, K)
    :rtype: int
    """
    return math.floor(((n * a) % 1)*K)


## Let's test the equidistribution of it, for ~5% K.

In [42]:
# Let's see where each (i, j, k) maps to (q)
index_map = np.zeros((10, 400, 352)) 

# KK is the range we want to hash to. Note the 0.05. 
KK = math.floor(10 * 400 * 352 * 0.05)

# Let's count how often each hash value pops up.
reverse_index_map = [[] for _ in range(KK)]
total_qqs = np.zeros(KK)

start = time.perf_counter()

for ii in range(10):
  for jj in range(400):
    for kk in range(352):
      xx = np.ravel_multi_index((ii, jj, kk), (10, 400, 352))
      qq = equi_hash(xx, K = KK)
      
      # count for ijk
      index_map[ii][jj][kk] = qq

      # count for q
      reverse_index_map[qq].append([ii, jj, kk])
      total_qqs[qq] += 1

end = time.perf_counter()

print(f"Calculated {KK} hashes in {end - start:.2f} seconds.")


Calculated 70400 hashes in 9.64 seconds.


### Before testing distribution, let's optimize the hash...

10 Seconds is slow! What gives?

In [43]:
# Let's just test the hash time, ignoring mapping...
start = time.perf_counter()

for ii in range(70400):
      qq = equi_hash(xx, K = KK)

end = time.perf_counter()

print(f"Calculated {KK} hashes in {end - start:.2f} seconds.")

# ?!? Okay, the hash is very fast.

Calculated 70400 hashes in 0.18 seconds.


In [44]:
start = time.perf_counter()

for ii in range(10):
  for jj in range(400):
    for kk in range(352):
      xx = np.ravel_multi_index((ii, jj, kk), (10, 400, 352))
      qq = equi_hash(xx, K = KK)

end = time.perf_counter()

print(f"Calculated {KK} hashes in {end - start:.2f} seconds.")

# Wow! `np.ravel_multi_index` is SLOW!

Calculated 70400 hashes in 7.00 seconds.


In [45]:
def ravel_multi_index(
    index = (0, 0, 0),
    shape = (10, 400, 352)
):
    """Flatten an index for a given range

    :param index: Multi-axis index, defaults to (0, 0, 0)
    :type index: tuple, optional
    :param shape: Range of the index, defaults to (10, 400, 352)
    :type shape: tuple, optional
    :return: Flat index for the given shape
    :rtype: int
    """
    assert len(index) == len(shape)
    flat_index = 0
    for axis in range(len(index)):
        flat_index = flat_index * shape[axis]
        flat_index = flat_index + index[axis]
    
    return flat_index


In [46]:
start = time.perf_counter()

for ii in range(10):
  for jj in range(400):
    for kk in range(352):
      xx = ravel_multi_index((ii, jj, kk), (10, 400, 352))
      qq = equi_hash(xx, K = KK)

end = time.perf_counter()

print(f"Calculated {KK} hashes in {end - start:.2f} seconds.")

# Wow! `np.ravel_multi_index` is SLOW! We speed it up a lot by replacing it here.

Calculated 70400 hashes in 1.43 seconds.


### Okay, now let's actually analyze it.

In [34]:
# index_map[i][j][k] -> q
# reverse_index_map[k] -> list of [i, j, k]
# total_qqs[k] -> total ii

In [40]:
def describe(array, axis = None):
    return f"""
Min:\t{array.min(axis = axis)}
Max:\t{array.max(axis = axis)}
Std:\t{array.std(axis = axis)}
Mean:\t{array.mean(axis = axis)}
    """

print("Distribution of q indices")
print(describe(total_qqs))


Min:	19.0
Max:	22.0
Std:	0.5737882488736193
Mean:	20.0
    


In [54]:
for total in [19, 20, 21, 22]:
    print(f"There are {sum(total_qqs == total):>5} indices mapping to {total} qq")

There are 11184 indices mapping to 19 qq
There are 48437 indices mapping to 20 qq
There are 10374 indices mapping to 21 qq
There are   405 indices mapping to 22 qq


This is great! Because we reduce to 5%, we expect exactly on average to have 20 ijk indices per q value.

Furthermore, this distribution is incredibly tight.